In [0]:
import os
import zipfile
import random
import math
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

import warnings

warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

In [0]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

In [0]:
try:
#   for path in ['/tmp/cats-v-dogs/training/cats', '/tmp/cats-v-dogs/training/dogs', '/tmp/cats-v-dogs/testing/cats', '/tmp/cats-v-dogs/testing/dogs']:
#     for file in os.listdir(path):
#       os.remove(os.path.join(path, file))
#     print(len(os.listdir(path)))
  
  os.makedirs('/tmp/cats-v-dogs/training/cats')
  os.makedirs('/tmp/cats-v-dogs/training/dogs')
  os.makedirs('/tmp/cats-v-dogs/testing/cats')
  os.makedirs('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

In [0]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  list_of_files = os.listdir(SOURCE)
  nbr_of_files = len(list_of_files)
  index = random.sample([x for x in range(nbr_of_files)], nbr_of_files)
  split_limit = math.ceil(nbr_of_files*SPLIT_SIZE)
  train_idx = index[:split_limit]
  test_idx = index[split_limit:]
  
  for idx in train_idx:
    filename = list_of_files[idx]
    path = os.path.join(SOURCE, filename)
    if os.path.getsize(path) == 0:
      print(f'{filename} is zero length, so ignoring')
      continue
    copyfile(path, os.path.join(TRAINING, filename))
    
  for idx in test_idx:
    filename = list_of_files[idx]
    path = os.path.join(SOURCE, filename)
    if os.path.getsize(path) == 0:
      print(f'{filename} is zero length, so ignoring')
      continue
    copyfile(path, os.path.join(TESTING, filename))


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [0]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

In [0]:
datagen_params = {
    'rescale': 1./255,
    'rotation_range': 40,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'shear_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True,
    'fill_mode': 'nearest'
}

TRAINING_DIR = '/tmp/cats-v-dogs/training'
train_datagen = ImageDataGenerator(**datagen_params)
train_generator = train_datagen.flow_from_directory(
                                  TRAINING_DIR,
                                  target_size=(150, 150),
                                  batch_size=100,
                                  class_mode='binary')

VALIDATION_DIR = '/tmp/cats-v-dogs/testing'
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
                                  VALIDATION_DIR,
                                  target_size=(150, 150),
                                  batch_size=100,
                                  class_mode='binary')

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [0]:
class StopTrainingCallback(tf.keras.callbacks.Callback):
  
  def __init__(self):
    tf.keras.callbacks.Callback.__init__(self)
    self._threhold = .2
  
  def on_epoch_end(self, epoch, logs=None):
    
    if logs.get('val_acc') >= .95:
      print('\nReached .95 of val_acc so stopping training')
      self.model.stop_training = True
      self.model.save_weights('m95.h5')
    elif logs.get('val_acc') > .94:
      self.model.save_weights('m94.h5')
    elif logs.get('val_acc') > .93:
      self.model.save_weights('m93.h5')
    elif logs.get('val_acc') > .92:
      self.model.save_weights('m92.h5')
    elif logs.get('val_acc') > .91:
      self.model.save_weights('m91.h5')
    elif logs.get('val_acc') > .90:
      self.model.save_weights('m90.h5')

In [0]:
history = model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1,
                              callbacks=[StopTrainingCallback()],
                              validation_data=validation_generator)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(history.history['acc'], color='r', label="Training Accuracy")
plt.plot(history.history['val_acc'], color='b', label="Validation Accuracy")
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], color='r', label="Training Loss") # , "Training Loss"
plt.plot(history.history['val_loss'], color='b', label="Validation Loss") # , "Validation Loss"
plt.title('Training and validation loss')
plt.legend()
plt.show()